코랩쓰고나서 암이 나았습니다.

여기서는 램 12기가까지 지원하는데 얼마나 썼는지 계속 확인할 수 있어서 확인해보니

이제까지 몇십시간씩 걸린건 역시 램이 딸려서 작동을 멈춰버린 상태여서 그런거였네요

여기서도 20만개 한번에 돌리니까 메모리 에러 나서 5만개씩 쪼개서 4번 돌려야겠어요

메모리 문제때문에 pickle로 내렸다가 올렸다가 반복하면서 했습니다

요약하면 ast.literal_eval => convert token to id => padding만 했고 이제 kobert로 embedding 단계에요.

```
pip install kobert-transformers
```


```
import pickle
import numpy as np
import pandas as pd
import ast
import time

import csv
import torch
import tensorflow as tf

from kobert_transformers import get_tokenizer
from kobert_transformers import get_kobert_model
```


```
# 구글 드라이브 마운트 시키기

import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)
```



```
cd /content/mnt/My Drive/Colab Notebooks
```

```
data = pd.read_csv('news_tokenized.csv')
data.info()
```


# body_morphs에 ast.literal_eval 시켜주기 => token 파일

body_morphs_list_0.pkl

body_morphs_list_1.pkl

body_morphs_list_2.pkl

body_morphs_list_3.pkl

파일 생성

```
body_morphs_list_0 = []

start = time.time()
for i in range(len(data.body_morphs[:50000])):
    body_morphs_list_0.append(ast.literal_eval(data.body_morphs[i]))
    now = time.time()
    if i % 10000 == 0:
      print('{}번째 작업, 걸린 시간 : {}분 {}초'.format(i,
                                        round((now - start) // 60),
                                        round((now - start) % 60)))
      
with open('body_morphs_list_0.pkl', 'wb') as file:
  pickle.dump(body_morphs_list_0, file)
```


위에 cell 돌아가는데 2분 걸림

```
body_morphs_list_1 = []

for element in data.body_morphs[50000:100000]:
    body_morphs_list_1.append(ast.literal_eval(element))

with open('body_morphs_list_1.pkl', 'wb') as file:
  pickle.dump(body_morphs_list_1, file)
```


```
body_morphs_list_2 = []

for element in data.body_morphs[100000:150000]:
    body_morphs_list_2.append(ast.literal_eval(element))

with open('body_morphs_list_2.pkl', 'wb') as file:
  pickle.dump(body_morphs_list_2, file)
```


```
body_morphs_list_3 = []

for element in data.body_morphs[150000:]:
    body_morphs_list_3.append(ast.literal_eval(element))

with open('body_morphs_list_3.pkl', 'wb') as file:
  pickle.dump(body_morphs_list_3, file)
```


# 다시 읽어들여서 token을 token id로 변환

bert의 maximum sequence length가 512이므로 token id를 512번째까지만 저장

512 미만으로 token length를 가지고 있는 데이터는 뒤에서 zero padding을 해줄 예정

Hannanum으로 tokenizing을 해서 kobert_tokenizer가 인식 못하는 token은 token_id가 0으로 변환되었음

```
tokenizer = get_tokenizer()
```


```
with open('body_morphs_list_0.pkl', 'rb') as file:
    body_morphs_list_0 = pickle.load(file)

body_morphs_token_id_0 = []

for element in body_morphs_list_0:
  element_id = tokenizer.convert_tokens_to_ids(element)
  body_morphs_token_id_0.append(element_id[:512])

with open('body_morphs_list_1.pkl', 'rb') as file:
    body_morphs_list_1 = pickle.load(file)

body_morphs_token_id_1 = []

for element in body_morphs_list_1:
  element_id = tokenizer.convert_tokens_to_ids(element)
  body_morphs_token_id_1.append(element_id[:512])
```


```
body_morphs_token_id_01 = body_morphs_token_id_0 + body_morphs_token_id_1

with open('body_morphs_token_id_01.pkl', 'wb') as file:
  pickle.dump(body_morphs_token_id_01, file)
```


```
with open('body_morphs_list_2.pkl', 'rb') as file:
    body_morphs_list_2 = pickle.load(file)

body_morphs_token_id_2 = []

for element in body_morphs_list_2:
  element_id = tokenizer.convert_tokens_to_ids(element)
  body_morphs_token_id_2.append(element_id[:512])

with open('body_morphs_list_3.pkl', 'rb') as file:
    body_morphs_list_3 = pickle.load(file)

body_morphs_token_id_3 = []

for element in body_morphs_list_3:
  element_id = tokenizer.convert_tokens_to_ids(element)
  body_morphs_token_id_3.append(element_id[:512])
```


```
body_morphs_token_id_23 = body_morphs_token_id_2 + body_morphs_token_id_3

with open('body_morphs_token_id_23.pkl', 'wb') as file:
  pickle.dump(body_morphs_token_id_23, file)
```


# token_id 불러와서 kobert_embedding 작업

```
with open('body_morphs_token_id_01.pkl', 'rb') as file:
  body_morphs_token_id_01 = pickle.load(file)

with open('body_morphs_token_id_23.pkl', 'rb') as file:
  body_morphs_token_id_23 = pickle.load(file)

body_token_id = body_morphs_token_id_01 + body_morphs_token_id_23
```


```
tokenizer = get_tokenizer()

pad = tokenizer.pad_token_id

print(pad) # 출력값 : 1

dir(tokenizer)
```


# zero padding

```
# 길이가 512 미만인 애들 뒤쪽에 padding
# 0은 tokenizer의 convert_token_to_id가 모르는 단어, 1은 패딩

body_token_id_padding = []

for element in body_token_id:
  if len(element) < 512:
    pad_element = element + [pad] * (512 - len(element))
  else:
    pad_element = element

  body_token_id_padding.append(pad_element)

with open('body_token_id_padding.pkl', 'wb') as file:
  pickle.dump(body_token_id_padding, file)
```


In [1]:
pip install kobert-transformers

In [2]:
import pickle
import numpy as np
import pandas as pd
import ast
import time

import csv
import torch
import tensorflow as tf

from kobert_transformers import get_tokenizer
from kobert_transformers import get_kobert_model

In [3]:
cd /content/mnt/My Drive/Colab Notebooks

/content/mnt/My Drive/Colab Notebooks


In [4]:
with open('body_token_id_padding.pkl', 'rb') as file:
  body_token_id_padding = pickle.load(file)

In [5]:
model = get_kobert_model()

# embedding 단계

200000개를 해야되는데 10개씩밖에 변환 못하는거 실화인가

In [7]:
body_sequence_output_0, body_pooled_output_0 = model(torch.LongTensor(body_token_id_padding[:10]))

In [8]:
body_sequence_output_0.shape

torch.Size([10, 512, 768])